<a href="https://colab.research.google.com/github/alexacheui/ateema_capstone/blob/main/typeform_api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install flask pyngrok

from flask import Flask, request
from pyngrok import ngrok

app = Flask(__name__)

@app.route("/")
def index():
    return "<h1>Hello World</h1>"

@app.route('/webhook', methods=['POST'])
def typeform_webhook():
    print(request.json)
    request_data = request.get_json()
    # print(request_data['form_response']['answers'][0]['choice']['label'])
    # print(request_data)
    return 'Typeform webhook'

if __name__ == '__main__':
  ngrok.set_auth_token("2hw9nVAW8Ib9JMw2HszGHTWfnqv_2qfZMpkPx6fLsNv93ZCCx")
  ngrok_tunnel = ngrok.connect(5000)
  print('Public URL:', ngrok_tunnel.public_url)
  app.run(debug=True)

Public URL: https://d821-104-154-173-29.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [16/Jun/2024 20:32:36] "POST /webhook HTTP/1.1" 200 -


{'event_id': '01J0HC0WJD9WAWEGEVKZHN4BD2', 'event_type': 'form_response', 'form_response': {'form_id': 'FkcBezxr', 'token': '01J0HC0WJD9WAWEGEVKZHN4BD2', 'landed_at': '2024-06-16T20:32:36Z', 'submitted_at': '2024-06-16T20:32:36Z', 'definition': {'id': 'FkcBezxr', 'title': 'AI Video Survey', 'fields': [{'id': 'cyC2jup2h9qd', 'ref': '596e32db-f5ef-422a-85b0-1e64723cff81', 'type': 'short_text', 'title': 'First name', 'properties': {}}, {'id': 'Ap9DQevjbydK', 'ref': '8b4f0707-c312-4658-94b0-d215a63fced2', 'type': 'short_text', 'title': 'Last name', 'properties': {}}, {'id': 'eZm0OtqIP460', 'ref': '600b2743-94a4-49bd-a9e9-2651313a9c5f', 'type': 'phone_number', 'title': 'Phone number', 'properties': {}}, {'id': 'RWH1jVaQrbdz', 'ref': 'dfe478d9-e2e2-4e75-9623-6280b9567724', 'type': 'email', 'title': 'Email', 'properties': {}}, {'id': 'rAKa2WQMixJL', 'ref': '86a2d1bb-5371-47bd-9e59-8556125ea162', 'type': 'picture_choice', 'title': '*Travel is always exciting! which town are you visiting? *', '

INFO:werkzeug:127.0.0.1 - - [16/Jun/2024 20:32:46] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [16/Jun/2024 20:32:46] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [16/Jun/2024 20:32:51] "GET /webhook HTTP/1.1" 405 -
